<a href="https://colab.research.google.com/github/ETorresSacha/Analisis-de-datos/blob/main/determinacio_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import io
import os
from google.colab import files
import seaborn as sns
from sklearn import  preprocessing
import random

In [ ]:
# Descargar el archivo
# Guardar en Excel
data.to_excel("data7.xlsx", index=False)
files.download('data7.xlsx')

In [2]:
uploaded = files.upload()

Saving rul_potenciaa.xlsx to rul_potenciaa.xlsx


In [ ]:
# para borrar los datos que se subió
os.remove("CAEX_RUL4.xlsx")

In [ ]:
data = pd.read_excel(io.BytesIO(uploaded['data_ultimo.xlsx']))

In [ ]:
# vamos a crear los ciclos por cada equipo

# Ordenamos primero por equipo y fecha
data = data.sort_values(by=['equipment','fecha_telemetria'])

# Definimos el inicio de un nuevo ciclo cuando:
# - cambia el equipo
# - o el RUL vuelve a subir (después de llegar a 0)
data["new_cycle"] = (
    (data["equipment"] != data["equipment"].shift()) |   # cambio de equipo
    (data["RUL_Horas"] > data["RUL_Horas"].shift())            # RUL "reinicia"
).astype(int)

# Ahora creamos un contador acumulativo de ciclos dentro de cada equipo
data["Ciclo"] = data.groupby("equipment")["new_cycle"].cumsum()

# Eliminamos la columna auxiliar
data = data.drop(columns=["new_cycle"])

# Mover columna Ciclo a la posición 1
col = data.pop("Ciclo")            # sacar la columna
data.insert(5, "Ciclo", col)       # insertarla en la posición 5

In [ ]:
data = data.drop(columns=['rn'])

In [ ]:
# Seleccionamos los equipos mas relevantes (3 equipos)
data= data[data["equipment"].isin(["C314","C311","C332"])]

In [ ]:
data.reset_index(drop=True)

In [ ]:
# Reemlazamos los nulos por ceros
data = data.fillna(0)

In [ ]:
data =data.iloc[:,3:]
data.reset_index(drop=True)

In [ ]:
# ÚTIL PARA CUANDO EVALUAMOS E COMPORTAMIENTO DE LAS VARIABLES RESPECTO A CADA EQUIPO, Y ASI DETERMINAR LOS EQUIPOS CON MEJORES DATOS

# calcularemos la importancia de las variables (features) ---! esto es útil pero lo colocaremos en el orden que corresponde cuando se arme el archivo final !!!!!!
from sklearn.ensemble import RandomForestRegressor

X = data.drop(columns=["RUL_Horas"])
y = data["RUL_Horas"]
X = data[['RUL_Horas','horometro','Ciclo','Eng_Oil_Press_Lo_Idle_Min','Eng_Oil_Press_Max','Eng_Oil_Temp_Max','Engine_Coolant_Temp_Max','Eng_Oil_Press_Hi_Idle_Min']]
#X = data[['RUL_Horas','horometro','Ciclo','Ambient_Temp_Max','Boost_RB_Press_Max','Exhaust_Temp_Max_LF'	,'Exhaust_Temp_Max_LR','Steering_Press_Max','Blow_by_Press_Max']]
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X, y)

importancess = pd.DataFrame({
    "feature": X.columns,
    "importance": model.feature_importances_
}).sort_values(by="importance", ascending=False)

importancess.head(20)

# DESPUÉS DE EVALUAR LOS DATOS DE CADA EQUIPO, NOS QUEDAMOS CON LOS EQUIPOS: 314,311,332 EN ESE ORDEN

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# Definimos las variables independientes (sin incluir RUL_Horas porque es el target)
X = data[['equipment','horometro','Ciclo',
          'Eng_Oil_Press_Lo_Idle_Min','Eng_Oil_Press_Max',
          'Eng_Oil_Temp_Max','Engine_Coolant_Temp_Max',
          'Eng_Oil_Press_Hi_Idle_Min']]

# Variable objetivo
y = data["RUL_Horas"]

# Entrenar el modelo
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X, y)

# Calcular importancia de las variables
importancess = pd.DataFrame({
    "feature": X.columns,
    "importance": model.feature_importances_
}).sort_values(by="importance", ascending=False)

print(importancess.head())


In [ ]:
# ANALIZAMOS EL COMPORAMIENTO DE LAS METRICAS POR CADA VARIABLE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# columnas=['horometro', 'Ciclo', 'Engine_Oil_Filter_Differential_Pressure',
#        'Eng_Oil_Press_Lo_Idle_Min', 'Eng_Oil_Press_Max', 'Eng_Oil_Temp_Max',
#        'DC_Power_Max', 'Engine_Coolant_Temp_Max', 'Eng_Oil_Press_Hi_Idle_Min']

#columnas=['horometro', 'Ciclo','Eng_Oil_Temp_Max','Eng_Oil_Press_Max', 'Eng_Oil_Press_Lo_Idle_Min','Engine_Oil_Filter_Differential_Pressure']
columnas =['horometro','Ciclo','Ambient_Temp_Max','Boost_RB_Press_Max','Exhaust_Temp_Max_LF'	,'Exhaust_Temp_Max_LR','Steering_Press_Max','Blow_by_Press_Max']

# columnas=['horometro','Ciclo','Ambient_Temp_Max','Boost_RB_Press_Max','Exhaust_Temp_Max_LF',
#'Exhaust_Temp_Max_LR','Steering_Press_Max','Ambient_Temp_Ave','Blow_by_Press_Max','Eng_Oil_Temp_Max','Exhaust_Temp_Max_RR',
# 'Ambient_Temp_Min','Engine_Coolant_Temp_Max','Motor_B_Power_Ave','Brake_Oil_Temp_LF_Max','DC_Power_Max','Eng_Oil_Press_Hi_Idle_Min',
#'Hoist_Press2_Max',	'Engine_Oil_Filter_Differential_Pressure','Exhaust_Temp_Max_RF']

col_modelo = []
#y = data["RUL_Horas"].values
y =y.values
for col in columnas:
  col_modelo.append(col)
  data1 = data[col_modelo]
  data1 = pd.concat([data1,encoded_df],axis=1) # encoded_df, se encuentra en la parte superior -- esta concadenando con los valores de OneHotEncoder
  x = data1.values

  # dividir los valores para el entrenamiento y testear los datos
  x_train,x_test,y_train,y_test = train_test_split(x,y , test_size=0.2,  random_state=42)
  # llamamos a la regresión
  forest_reg = RandomForestRegressor(
          n_estimators=100,
          random_state=42,
          n_jobs=-1
  )
  # entrenamiento
  forest_reg.fit(x_train, y_train)
  #predicción
  y_pred = forest_reg.predict(x_test)
  r2 = r2_score(y_test,y_pred)
  print(col_modelo,r2)
